In [1]:
# !pip install sklearn torch datasets transformers

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
# cd /content/drive/MyDrive/research/code/toxic-comments/Scripts/

In [4]:
import pandas as pd

dataset_number = 2

# Lacafe
text = 'txt'
label = 'has_anger'
train_data = pd.read_csv(f'./Datasets/Lacafe/df_dataset_train_{dataset_number}.csv')
test_data = pd.read_csv(f'./Datasets/Lacafe/df_dataset_test_{dataset_number}.csv')

# Fortuna
# text = 'text'
# label = 'hatespeech_comb'
# train_data = pd.read_csv(f'./Datasets/Fortuna/2019-05-28_portuguese_hate_speech_binary_classification_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/Fortuna/2019-05-28_portuguese_hate_speech_binary_classification_test_{dataset_number}.csv')

# ToLD-BR
# text = 'text'
# label = 'hatespeech'
# train_data = pd.read_csv(f'./Datasets/ToldBR/ToLD-BR-Treated_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/ToldBR/ToLD-BR-Treated_test_{dataset_number}.csv')

# OffComBr
# text = 'text'
# label = 'offensive'
# train_data = pd.read_csv(f'./Datasets/OffComBR/OffComBR2_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/OffComBR/OffComBR2_test_{dataset_number}.csv')

In [5]:
from datasets import Dataset, DatasetDict

print(train_data)
train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)

raw_datasets = DatasetDict({'train': train_data, 'test': test_data})

train_data
test_data
raw_datasets

2022-05-03 11:31:47.166830: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-03 11:31:47.166863: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


                                                    txt  has_anger
0     >>57598o melhor cardo sempre é corrida, mas ca...          1
1     📽️\no army cantou o fanchat de magic shop um p...          0
2                                              ai gente          0
3     o amor não é monocromático. no mês do orgulho ...          0
4     @flexzonasulrec @meumonstrinho1 @ativorec @cau...          0
...                                                 ...        ...
5364  >>62310o mais engraçado é que mesmo com todas ...          1
5365  para mim ficou claro que nem o ministro moro, ...          0
5366  >>28828>>28829não tem como assistir a esse víd...          1
5367                          alguém: vc que sabe\neu:           0
5368  >além da suspeita de pagamento em dobro, a pro...          1

[5369 rows x 2 columns]


DatasetDict({
    train: Dataset({
        features: ['txt', 'has_anger'],
        num_rows: 5369
    })
    test: Dataset({
        features: ['txt', 'has_anger'],
        num_rows: 2302
    })
})

In [6]:
# Tratamento básico dos dados

train_texts = raw_datasets['train'][text]
train_labels = raw_datasets['train'][label]
test_texts = raw_datasets['test'][text]
test_labels = raw_datasets['test'][label]

print("TrainTexts Length: ", len(train_texts))
print("TrainLabels Length: ", len(train_labels))
print("TestTexts Length: ", len(test_texts))
print("TestLabels Length: ", len(test_labels))

TrainTexts Length:  5369
TrainLabels Length:  5369
TestTexts Length:  2302
TestLabels Length:  2302


In [7]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1)

In [8]:
print("TrainTexts Length: ", len(train_texts))
print("TrainLabels Length: ", len(train_labels))
print("ValidationTexts Length: ", len(val_texts))
print("ValidationLabels Length: ", len(val_labels))

TrainTexts Length:  4832
TrainLabels Length:  4832
ValidationTexts Length:  537
ValidationLabels Length:  537


In [9]:
print(test_labels[:15])

[0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0]


In [10]:
# Importando o Tokenizer

from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

In [11]:
# Tokenização dos datasets

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

# print(val_encodings)

In [12]:
# Voltando os datasets tokenizados para instâncias da classe de Dataset

import torch

class HSDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HSDataset(train_encodings, train_labels)
val_dataset = HSDataset(val_encodings, val_labels)
test_dataset = HSDataset(test_encodings, test_labels)

In [13]:
# print(train_dataset.labels)

In [14]:
from transformers import AutoModelForSequenceClassification

# model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "test_trainer",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
)

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset
)

In [17]:
import numpy as np
from datasets import load_metric

metric_acc = load_metric("accuracy")
metric_f1 = load_metric("f1")
metric_precision = load_metric("precision")
metric_recall = load_metric("recall")

def cma(labels, predictions):
    cm = {'true_positive': 0, 'true_negative': 0, 'false_positive': 0, 'false_negative': 0}
    for i in range(0, len(labels)):
        if labels[i] == predictions[i]:
            if labels[i] == 1:
                cm['true_positive'] += 1
            else:
                cm['true_negative'] += 1
        else:
            if labels[i] == 0:
                cm['false_positive'] += 1
            else:
                cm['false_negative'] += 1
    return cm

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    result = {
        'accuracy': metric_acc.compute(predictions=predictions, references=labels),
        'precision': metric_precision.compute(predictions=predictions, references=labels),
        'recall': metric_recall.compute(predictions=predictions, references=labels),
        'f1': metric_f1.compute(predictions=predictions, references=labels),
        'total': len(predictions),
        'cm': cma(labels, predictions)
    }
    
    return result

In [18]:
# Fine-tuning with HuggingFace
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()
trainer.evaluate(eval_dataset=test_dataset)

***** Running training *****
  Num examples = 4832
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 6040


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Total,Cm
1,0.196800,0.354868,{'accuracy': 0.9180633147113594},{'precision': 1.0},{'recall': 0.8232931726907631},{'f1': 0.9030837004405287},537,"{'true_positive': 205, 'true_negative': 288, 'false_positive': 0, 'false_negative': 44}"
2,0.151400,0.147926,{'accuracy': 0.9720670391061452},{'precision': 0.9717741935483871},{'recall': 0.9678714859437751},{'f1': 0.9698189134808854},537,"{'true_positive': 241, 'true_negative': 281, 'false_positive': 7, 'false_negative': 8}"
3,0.104600,0.121344,{'accuracy': 0.9795158286778398},{'precision': 0.983739837398374},{'recall': 0.9718875502008032},{'f1': 0.9777777777777777},537,"{'true_positive': 242, 'true_negative': 284, 'false_positive': 4, 'false_negative': 7}"
4,0.052800,0.115599,{'accuracy': 0.9813780260707635},{'precision': 0.9799196787148594},{'recall': 0.9799196787148594},{'f1': 0.9799196787148594},537,"{'true_positive': 244, 'true_negative': 283, 'false_positive': 5, 'false_negative': 5}"
5,0.039500,0.094411,{'accuracy': 0.9851024208566108},{'precision': 0.9878542510121457},{'recall': 0.9799196787148594},{'f1': 0.9838709677419354},537,"{'true_positive': 244, 'true_negative': 285, 'false_positive': 3, 'false_negative': 5}"


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 537
  Batch size = 4
Trainer is attempting to log a value of "{'accuracy': 0.9180633147113594}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 1.0}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8232931726907631}" of type <class 'dict'> for key "

Trainer is attempting to log a value of "{'true_positive': 244, 'true_negative': 285, 'false_positive': 3, 'false_negative': 5}" of type <class 'dict'> for key "eval/cm" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 2302
  Batch size = 4


Trainer is attempting to log a value of "{'accuracy': 0.9813205907906168}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9834205933682374}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.9791485664639444}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9812799303439269}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'true_positive': 1127, 'true_negative': 1132, 'f

{'eval_loss': 0.11433910578489304,
 'eval_accuracy': {'accuracy': 0.9813205907906168},
 'eval_precision': {'precision': 0.9834205933682374},
 'eval_recall': {'recall': 0.9791485664639444},
 'eval_f1': {'f1': 0.9812799303439269},
 'eval_total': 2302,
 'eval_cm': {'true_positive': 1127,
  'true_negative': 1132,
  'false_positive': 19,
  'false_negative': 24},
 'eval_runtime': 727.7116,
 'eval_samples_per_second': 3.163,
 'eval_steps_per_second': 0.792,
 'epoch': 5.0}